# Derivitives With Recursion

The process of finding the derivitive of a function uses recursion. For example take the function:


$f(x) \;= \;3(sin(x)x^{2})^{2}$

$f'(x) \;=\; 3*\frac{d}{dx}[(sin(x)x^{2})^{2}] \;=\; 3*2*(sin(x)x^{2})*\frac{d}{dx}[(sin(x)x^{2})]\; =\; 6*(sin(x)x^{2})*(sin(x)*\frac{d}{dx}[x^{2}]+\frac{d}{dx}[sin(x)]*x^{2})$

$f'(x) \;=\; 6*(sin(x)x^{2})*(sin(x)*2x+cos(x)*x^{2})$

For that derivitive you end up using the product rule within the chain rule so things can get very messy/complicated real quick. **For that reason it is important to not skip steps and use d/dx notation!**

Through this notebook I will be demonstrating how to use recursion in python to find derivitives of complicated nested functions. 

As there are different ways to type equations (e.g. 2x, 2*x and 2(x)), I needed to figure a way to standardize the input. So instead of using direct operations, the user would have to input a string in the following form:

input string = "sum(product(2,x),sin(x))"

This would be equivalent to: $2x + sin(x)$

The different operations that I chose were:
1. sum(a,b)     => a + b
2. product(a,b) => a * b
3. quot(a,b)    => a / b
4. exp(a,b)     => a ^ b
5. sin(a) 
6. sin(b)

There would have to be a way for the computer to recognize what function is being applied, specifically the outermost to provide a starting point. So if input = "sum(product(2,x),sin(x))", the computer knows that the outermost function is "sum. To do this I implemented the following function:

In [343]:
#if term is not a constant nor x then iterate through string until hit a "(.The function is the characters before
#                                                                                                          the "("
def funcType(term):
    if(term.isnumeric()):
        return "constant"
    if(term=="x"):
        return "x"
    termChar = ""
    ind = 0
    while(termChar!="("):   
        termChar = term[ind]
        ind+=1  
    return term[0:ind-1]

In [344]:
funcType("sum(product(2,x),sin(x))")

'sum'

Then the tough part is splitting the arguments. In the example above the arguments are 1.product(2,x) and 2.sin(x). The goal is to split this into a list. And you can't just split the string by commas because you would end up with 3 arguments. The code I used may be a little tricky to understand but it keeps track of the number of unmatched "(". If for example the number of open parenthesis were 2 then you would know that the curent term is nested. If the number of open parenthesis is 0 then there is no nesting going on. 

In [345]:
def getArgs(term):
    if(funcType(term)!="constant" and funcType(term)!="x"):
        term = term.replace(funcType(term),"",1)[1:-1]
    splits = [-1]
    isParenthesis = 0
    for i,char in enumerate(term):
        if(char=="("):
            isParenthesis += 1      # Gets the level
        elif(char==")"):
            isParenthesis -= 1
        elif(char=="," and isParenthesis==0):
            splits.append(i)
    splits.append(len(term))
    return [term[splits[i]+1:splits[i+1]].replace(" ","") for i in range(len(splits)-1)]

In [346]:
getArgs("sum(product(2,x),sin(x))")

['product(2,x)', 'sin(x)']

This allows the recursion to happen. der(a+b) = der(a) + der(b). Now those arguments can be plugged back into the derivitive function. So at this point der(product(2,x)) and der(sin(x)) will be evaluated then added together. 

Finally it is time to implement the derivitive function. This is where the recursion happens. Essentially is is just a whole bunch of if statements relative to the function being applied. So if a sum is plugged into the derivitive function, say sum(2,x), then it will return derivitive(2) + derivitive(x). Some better examples of the process are listed down below:



der("product(3,x)") => der(3)*x + 3*der(x) => 0 * x + 3 * 1

der("sum(product(3,x),7)") => der(product(3,x)) + der(7) => der(3)*x + 3*der(x) + 0 => 0 * x + 3 * 1 + 0

der("exp(prod(3,x),4)") => 4*(prod(3,x))^3 * der(prod(3,x)) => 4*(prod(3,x))^3 * ((3)*der(x)+der(3)*x)=> 4*(prod(3,x))^3 * (3 * 1 + 0 * x)

One important note is that you can use the product rule with trivial products such as $f(x)=3x$. It's second nature to find the derivitive for this function as its just 3. But it is totally applicable to use the product rule here. 
$f'(x) = 3*1+0*x = 3$


Anyways here is the code I used for the derivitive function:


In [347]:
def der(term):
    func = funcType(term)    #Outermost layer function
    args = getArgs(term)     #Arguments within outermost function 
    if(func=="cos"):
        return "(-sin("+args[0]+") * " + der(args[0])+")"
    
    elif(func=="sin"):
        return "(cos("+args[0]+") * " + der(args[0])+")"
    
    elif(func=="exp"):
        return "(("+args[1] + "*(" + args[0] + ")^" + str(int(args[1])-1) + ")*(" + der(args[0]) + "))"
    
    elif(func=="sum"):
        return "("+der(args[0])+" + "+der(args[1])+")"
    
    elif(func=="prod"):
        return "(("+args[0] + "*" +der(args[1]) + ") + (" +der(args[0]) + "*" +args[1]+"))"
    
    elif(func=="quot"):
        return "(("+args[1] + "*" +der(args[0]) + ") - (" + args[0] + "*" +der(args[1])+"))/("+args[1]+"^2)"
    
    elif(term == "x"):
        return "1"
    
    elif(term.isnumeric):
        return "0"
    

Lets try it out...

Let $f(x) = x+sin(x)$

Then $f'(x) = \frac{d}{dx}[x]+\frac{d}{dx}[sin(x)] = 1 + cos(x)$

In [348]:
der("sum(x,sin(x))")        

'(1 + (cos(x) * 1))'

Let $f(x) = (3x^{2}+5)^{4}$

Then $f'(x) = 4(3x^{2}+5)^{3}*\frac{d}{dx}[3x^{2}]$

$f'(x) = 4(3x^{2}+5)^{3}*6x$

In [349]:
der("exp(sum(prod(3,exp(x,2)),5),4)")

'((4*(sum(prod(3,exp(x,2)),5))^3)*((((3*((2*(x)^1)*(1))) + (0*exp(x,2))) + 0)))'

Notice how sometimes the raw string for functions are not evaluated? This can be fixed with the following code. Once again recursion is used:

In [350]:
def prettify(term):
    func = funcType(term)
    args = getArgs(term) 
    if(func=="sum"):
        return "(" + prettify(args[0]) + " + " + prettify(args[1]) + ")"
    if(func=="prod"):
        return "(" + prettify(args[0]) + "*" + prettify(args[1]) + ")"
    if(func=="quot"):
        return "(" + prettify(args[0]) + "/" + prettify(args[1]) + ")"
    if(func=="exp"):
        return "(" + prettify(args[0]) + "^" + prettify(args[1]) + ")"
    if(func=="cos" or func == "sin"):
        return func+"("+prettify(args[0])+")"
    else:
        return term
    

In [351]:
def der(term):
    func = funcType(term)    
    args = getArgs(term)     
    if(func=="cos"):
        return "(-sin("+prettify(args[0])+") * " + der(args[0])+")"
    
    elif(func=="sin"):
        return "(cos("+prettify(args[0])+") * " + der(args[0])+")"
    
    elif(func=="exp"):
        return "(("+args[1] + "*(" + prettify(args[0]) + ")^" + str(int(args[1])-1) + ")*(" + der(args[0]) + "))"
    
    elif(func=="sum"):
        return "("+der(args[0])+" + "+der(args[1])+")"
    
    elif(func=="prod"):
        return "(("+prettify(args[0]) + "*" +der(args[1]) + ") + (" +der(args[0]) + "*" +prettify(args[1])+"))"
    
    elif(func=="quot"):
        return "(("+prettify(args[1]) + "*" +der(args[0]) + ") - (" + prettify(args[0]) + "*" +der(args[1])+"))/("+args[1]+"^2)"
    
    elif(term == "x"):
        return "1"
    
    elif(term.isnumeric):
        return "0"
    

In [352]:
der("exp(sum(prod(3,exp(x,2)),5),4)")

'((4*(((3*(x^2)) + 5))^3)*((((3*((2*(x)^1)*(1))) + (0*(x^2))) + 0)))'

Now that looks a lot better although not perfect. I could find some way to fully simplify equations but that would be a whole new problem to work on. Lets try a few more...

Let $f(x) = \frac{3x^{2}}{sin(x)}$

Then $f'(x) = \frac{sin(x)*\frac{d}{dx}[3x^{2}] - 3x^{2}*\frac{d}{dx}[sin(x)]}{sin(x)^{2}}$

$f'(x) = \frac{sin(x)*6x-3x^{2}*cos(x)}{sin(x)^2}$

In [353]:
der("quot(prod(3,exp(x,2)) , sin(x) )")

'((sin(x)*((3*((2*(x)^1)*(1))) + (0*(x^2)))) - ((3*(x^2))*(cos(x) * 1)))/(sin(x)^2)'

Let $f(x) = sin((x+3)*(cos(x)^{2}))$

Then $f'(x) = cos((x+3)*(cos(x)^{2}))*\frac{d}{dx}[(x+3)*(cos(x)^{2})] = cos((x+3)*(cos(x)^{2}))*((x+3)\frac{d}{dx}[(cos(x)^{2})]+ \frac{d}{dx}[(x+3)]*cos(x)^{2})     $


$f'(x) = cos((x+3)*(cos(x)^{2}))*((x+3)2*(cos(x))*-sin(x) + 1*cos(x)^{2})$


In [354]:
der("sin(prod(sum(x,3),exp(cos(x),2)))")    

'(cos(((x + 3)*(cos(x)^2))) * (((x + 3)*((2*(cos(x))^1)*((-sin(x) * 1)))) + ((1 + 0)*(cos(x)^2))))'

So based on the few examples I tried, it looks like the derivitive function is behaving correctly! For those who have read through this, I hope you have been inspired to try to projects like this one. If there are any questions about anything just send me an email. Thank you for reading!